AI Sales Agent
The general structure is:
1. A workflow of Agent calls
2. An agent that can use a tool
3. An agent that can call on other agents (Tools and Handoffs)

In [28]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from datetime import datetime as dt
import os
import asyncio
import requests

#import sendgrid
#from sendgrid.helpers.mail import Mail, Email, To, Content

In [43]:
load_dotenv(override=True)

True

## Setup Pushover

In [61]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [ ]:
def push(message):
    now = dt.now().strftime("%Y-%m-%d %H:%M:%S")
    message += f"\n\nSent at {now}"
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}    
    response = requests.post(pushover_url, data=payload)
    
    print(f"Push response: {response.status_code} - {response.text}")

In [67]:

push(f"TEST: Instead of Twilio!!")

Push: TEST: Instead of Twilio!!

Sent at 2025-10-21 07:30:15
Push response: 200 - {"status":1,"request":"60662750-ecfe-466f-aadd-ff4f7537a2a9"}


## Part 1. Manual Agent Workflow
1. Create 3 prompts
2. Create 3 agents
3. Run the code
3.1 Streaming - Use of stream to return results. Very nice
3.2 ?
3.3 ?

In [39]:
# 1. 3x prompts
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

# 2. 3x agents
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [40]:
# 3.1 Streaming: Run the code with streaming
#
# "async for" returns a coroutine that yields events as they come in. In the next line we iterate through the responses
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Ensure Your SOC 2 Compliance Effortlessly with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to streamline your SOC 2 compliance efforts and prepare you for audits with ease.

In today’s regulatory landscape, ensuring compliance can be a daunting task that diverts your team’s focus from core business objectives. ComplAI simplifies this process by leveraging AI technology to automate documentation, monitor compliance status in real-time, and provide actionable insights tailored to your organization’s needs.

Here are a few key benefits of using ComplAI:

- **Automated Documentation**: Reduce manual effort and minimize errors with our smart documentation tools.
- **Real-Time Monitoring**: Keep abreast of your compliance status without the constant manual checks.
- **Audit Preparedness**: Be audit-ready at any moment, empowering your team to focus on growth.

I would love 

In [ ]:
# 3.2 Parallel execution of agents, using asyncio.gather

message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

In [42]:
# 3.3 A wokflow pattern

sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

message = "Write a cold sales email"

# Call three agents in parallel to generate cold sales emails
with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    # Call a fourth agent to pick the best email
    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Let’s Make Audits a Breeze 🍃

Hey [Recipient's Name],

I hope this email finds you in good spirits and even better compliance! 

Let’s talk about SOC2 audits. You know, those delightful moments when you realize you need to find 27 missing documents, decipher cryptic logs, and pray to the Compliance Gods? What if I told you that with ComplAI, you could swap chaos for comfort?

Think of us as your friendly AI sidekick—no cape required—ensuring your SOC2 compliance is as smooth as your morning coffee. ☕️ Our platform not only automates the paperwork but even prepares you for audits like a pro at a spelling bee—you’ll be ready to impress!

Curious? Let’s chat! I promise to sprinkle in some humor and keep the jargon to a minimum (because who needs that headache?).

Looking forward to hearing from you!

Best,  
[Your Name]  
[Your Position]  
ComplAI  
P.S. If compliance were a dance, we’d have all the right moves! 🕺💃


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: Use of tools
- Recall that we have three agents: sales_agent1, sales_agent2, sales_agent3.

In [134]:
# Examine one of the agents
agents = [sales_agent1, sales_agent2, sales_agent3]
[print(agent) for agent in agents]

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)
Agent(name='Engaging Sales Agent', instructions='You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and prepa

[None, None, None]

In [143]:
#  Create a tool using the @function_tool decorator

@function_tool
def push_message(msg: str) -> Dict[str, str]:
    #now = dt.now().strftime("%Y-%m-%d %H:%M:%S")
    #message += f"\n\nSent at {now}"
    
    if 1 == 0:
        print(f"push_message(): \n{msg}\n")
        return {"status": "success"}
    else:
        payload = {"user": pushover_user, "token": pushover_token, "message": msg}

        response = requests.post(pushover_url, data=payload)

        if response.status_code == 200:
            return {"status": "success"}
        else:
            return {"status": "failure"}

In [93]:
msg = f"Hello, this is a test message. \n{dt.now().strftime('%Y-%m-%d %H:%M:%S') }"
if 1 == 0:
    print(push_message(msg))

In [94]:
# Due to the decorator [@function_tool], the function is now also a tool that can be used by agents
# Let's view its metadata
push_message

FunctionTool(name='push_message', description='', params_json_schema={'properties': {'message': {'title': 'Message'}}, 'required': ['message'], 'title': 'push_message_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000014EC80D0E00>, strict_json_schema=True, is_enabled=True)

We can also convert an agent into a tool! 

This creates a wrapper around the agent, such that if the tool is called, it will call agent and trigger the call to the underlying LLM.

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a scold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000014EC8037CE0>, strict_json_schema=True, is_enabled=True)

In [136]:
# Generate three tools from the three sales agents and add them to an array of tools
description="Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

if 1 == 1: tools = [tool1, tool2, tool3, push_message]
else: tools = [tool1, push_message]
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000014EC74E3C40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000014EC7A86700>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

### Create the Sales Manager (the planning agent)

In [141]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these sequential steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Call each sales_agent exactly once. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email, using your judgment of which one is most effective.
 
3. Use the push_message tool to send the best email (and only the best email) to the user.

Crucial Rules:
- You must use all the sales_agent tools once each to generate the drafts — do not write them yourself.
- You must send ONE email using the push_message tool — never more than one.
"""

sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

In [142]:
# run the sales manager with tracing
with trace("Sales managerX.2"):
    result = await Runner.run(sales_manager, message)

#### Remember to check the trace for "Sales manager"
https://platform.openai.com/traces